In [23]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt!"

def getAllPlayerNames():
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    c.execute('SELECT player_name FROM player')
    playerNames = c.fetchall()
    conn.close()
    return playerNames



In [24]:
#needed to handle "thomas mueller" and "mueller thomas"
def getReverseName(name):
    x = name.find(" ")
    if x > -1:
        return name[x+1:] + " "+ name[:x]
    return "XXX"

#returns the database id of the player by name
def getPlayerIDByName(name):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    queryText = 'SELECT id FROM player WHERE player_name = "'+ name+'"'
    c.execute(queryText)
    playerID = c.fetchall()
    conn.close()
    
    return playerID[0][0]

#Returns list with names of found players
def getPlayersInQuery(query):
    playerList = list()
    for name in getAllPlayerNames():
        if  name[0].find(" ") > -1 and name[0] in query or getReverseName(name[0]) in query:
            playerList.append(name[0])
    return playerList



In [25]:
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    #query.replace("\xD6", "Oe")
    #query.replace("\xF6", "oe")

    #query.replace("\xAE", "Ae")
    #query.replace("\xE4", "ae")
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    print(query)
    return query
    
    
query1 = preProcessQuery(query1)

for element in getPlayersInQuery(query1):
    print(getPlayerIDByName(element))
    #print(type(getPlayerIDByName(element)))

Mueller Thomas ist der beste Spieler der Welt!
10243
